In [42]:
# coordinate positions for scraping from different resolutions

fastRecordIconPositionsX = [1136, 1171, 1207, 1244, 1280, 1316, 1352, 1388, 1424, 
1424, 
1424, 1424, 1352, 1316, 1280, 1244, 1207, 1171, 1136, 
1136, 1171, 1207, 1244, 1280, 1316, 1352, 1388]

fastRecordIconPositionsY = [611, 611, 611, 611, 611, 611, 611, 611, 611, 
637, 
677, 677, 677, 677, 677, 677, 677, 677, 677,
637, 637, 637, 637, 637, 637, 637, 637]

'''920, 507
952, 507
992, 507
1032, 507
1062, 507
1102, 507
1132, 507
1132, 550
1102, 550
1062, 550
1032, 550
992, 550
952, 550
920, 550
920, 585
952, 585
992, 585
1032, 585
1062, 585
1102, 585
1132, 585
1132, 623
992, 623
952, 623
920, 623'''

list1 = [920, 952, 992, 1032, 1062, 1102, 1132, 1132, 1102, 1062, 1032, 992, 952, 920, 920, 952, 992, 1032, 1062, 1102, 1132, 1132, 992, 952, 920]
list2 = [507, 507, 507, 507, 507, 507, 507, 550, 550, 550, 550, 550, 550, 550, 585, 585, 585, 585, 585, 585, 585, 623, 623, 623, 623]

assert len(fastRecordIconPositionsX) == len(fastRecordIconPositionsY)

assert len(list1) == len(list2)

# List of Positions for each record icon x, y (at given GUI scale, resolution (4k), window size) 
_4kRecordIconPositionsX = [1842, 2274, 2274, 1842, 1842, 2274]
_4kRecordIconPositionsY = [1155, 1155, 1209, 1209, 1260, 1260]

_2kRecordIconPositionsX = [921, 1137, 1137, 921, 921, 1137]
_2kRecordIconPositionsY = [int(i) / 2 for i in _4kRecordIconPositionsY]

fastRecordIconPositionsX = [1136, 1171, 1207, 1244, 1280, 1316, 1352, 1388, 1424, 
1424, 
1424, 1388, 1352, 1316, 1280, 1244, 1207, 1171, 1136, 
1136, 1171, 1207, 1244, 1280, 1316, 1352, 1388]

fastRecordIconPositionsY = [611, 611, 611, 611, 611, 611, 611, 611, 611, 
637, 
677, 677, 677, 677, 677, 677, 677, 677, 677,
637, 637, 637, 637, 637, 637, 637, 637]

assert len(fastRecordIconPositionsX) == len(fastRecordIconPositionsY)

# create a list of tuples from the 2k lists
recordIconPositions = list(zip(_2kRecordIconPositionsX, _2kRecordIconPositionsY))

# create a list of tuples from the 4k lists
_4krecordIconPositions = list(zip(_4kRecordIconPositionsX, _4kRecordIconPositionsY))

# create an extended list with values intrapolated in between the given values
recordIconPositionsExtended = []
for i in range(0, 5):
    recordIconPositionsExtended.append(recordIconPositions[i])
    recordIconPositionsExtended.append(((recordIconPositions[i][0] + recordIconPositions[i+1][0]) / 2, (recordIconPositions[i][1] + recordIconPositions[i+1][1]) / 2))

fastIconPositions = list(zip(list1, list2))

# Scale factors
scale_factor_x = 1920 / 2560
scale_factor_y = 1080 / 1440

# Original 2K coordinates
_2kRecordIconPositionsX = [921, 1137, 1137, 921, 921, 1137]
_2kRecordIconPositionsY = [577, 577, 604, 604, 630, 630]

# Calculate 1080p coordinates
_1080pRecordIconPositionsX = [int(x * scale_factor_x) for x in _2kRecordIconPositionsX]
_1080pRecordIconPositionsY = [int(y * scale_factor_y) for y in _2kRecordIconPositionsY]

# Create list of tuples for 1080p coordinates
recordIconPositions1080p = list(zip(_1080pRecordIconPositionsX, _1080pRecordIconPositionsY))

# Calculate extended positions for 1080p
recordIconPositionsExtended1080p = []
for i in range(0, 5):
    recordIconPositionsExtended1080p.append(recordIconPositions1080p[i])
    next_pos = ((recordIconPositions1080p[i][0] + recordIconPositions1080p[i+1][0]) // 2, 
                (recordIconPositions1080p[i][1] + recordIconPositions1080p[i+1][1]) // 2)
    recordIconPositionsExtended1080p.append(next_pos)

### Assembling Playerlist ###

Assembling playerlist from files on the Desktop. Adding players manually is possible here.

In [43]:
import requests
import json
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

def get_uuid_from_username(username):
    """Get UUID for a given username"""
    url = f"https://api.mojang.com/users/profiles/minecraft/{username}"
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            profile = response.json()
            return profile['id']
        else:
            return None
    except Exception as e:
        print(f"Error for username {username}: {e}")
        return None

def add_uuids_from_names(names_list, output_file=None, max_workers=10, delay=0.1):
    """Add UUIDs for a list of usernames"""
    
    if isinstance(names_list, str):
        with open(names_list, 'r') as f:
            names = [line.strip() for line in f if line.strip()]
    else:
        names = names_list
    
    print(f"Processing {len(names)} usernames...")
    
    name_to_uuid = {}
    failed_names = []
    
    def fetch_with_delay(username):
        time.sleep(delay)
        uuid = get_uuid_from_username(username)
        return username, uuid
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_name = {executor.submit(fetch_with_delay, name): name for name in names}
        
        for i, future in enumerate(as_completed(future_to_name)):
            username, uuid = future.result()
            if uuid:
                name_to_uuid[username] = uuid
                print(f"Progress: {i+1}/{len(names)} - {username}: {uuid}")
            else:
                failed_names.append(username)
                print(f"Progress: {i+1}/{len(names)} - Failed to get UUID for {username}")
    
    if output_file:
        with open(output_file, 'w') as f:
            json.dump(name_to_uuid, f, indent=2)
        print(f"UUID data saved to: {output_file}")
    
    print(f"\nProcess complete!")
    print(f"Successful: {len(name_to_uuid)}")
    print(f"Failed: {len(failed_names)}")
    
    if failed_names:
        print(f"Failed usernames: {', '.join(failed_names)}")
    
    return name_to_uuid, failed_names

In [44]:
if __name__ == "__main__":
    # Example usage with a list
    test_names = ["Otronix", "AnzeigeGehtRaus", "xIForgotMySkill", "kfc_man069", "SwLegende", "Bildungsarten", "linastunna", "CladyNoClip_" , "Nor_Malo", "zLachs", "sirmigorius", "Joel_the_king", "Nor_Malo"]
    
    # Or from a file:
    # result, failed = add_uuids_from_names('player_names.txt', 'new_player_uuids.json')
    
    result, failed = add_uuids_from_names(test_names, 'player_uuids_2.json')
    
    print(f"\nResults:")
    for name, uuid in result.items():
        print(f"  {name}: {uuid}")

Processing 13 usernames...
Progress: 1/13 - AnzeigeGehtRaus: 6c98cec55d4f4696825f55df358b47e5
Progress: 2/13 - Otronix: 072ea26fab9149eaac98660626a16515
Progress: 3/13 - zLachs: a1a2542d096d442d89d9c310f105986d
Progress: 4/13 - xIForgotMySkill: 3a09d83192cb4da6b61913ec91da3029
Progress: 5/13 - CladyNoClip_: 5f94cff616714fb0acaae254f26168f1
Progress: 6/13 - SwLegende: 6018fa9504ed40ecb7a99d080d4aafda
Progress: 7/13 - linastunna: d31e312b176248839effc988a340b5bf
Progress: 8/13 - Nor_Malo: 827c829f0f934174970cbb7a3de9ae5d
Progress: 9/13 - kfc_man069: 298151fbccbe42989c9ec501daefc0ca
Progress: 10/13 - Bildungsarten: 621cb3d46cb0402c8d423b1f4a183211
Progress: 11/13 - Joel_the_king: 79bc0af767d3497199ff8e5509527de3
Progress: 12/13 - Nor_Malo: 827c829f0f934174970cbb7a3de9ae5d
Progress: 13/13 - sirmigorius: a9207a414e2141adb14f919939696c4a
UUID data saved to: player_uuids_2.json

Process complete!
Successful: 12
Failed: 0

Results:
  AnzeigeGehtRaus: 6c98cec55d4f4696825f55df358b47e5
  Otronix:

In [45]:
import requests
import json
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import os

def get_username_from_uuid(uuid):
    """Get current username for a given UUID"""
    url = f"https://sessionserver.mojang.com/session/minecraft/profile/{uuid}"
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            profile = response.json()
            return profile['name']
        else:
            return None
    except Exception as e:
        print(f"Error for UUID {uuid}: {e}")
        return None

def update_player_names(input_file, output_file, max_workers=10, delay=0.1):
    """Update player names from UUIDs and save to new file"""
    
    # Load existing data
    with open(input_file, 'r') as f:
        old_data = json.load(f)
    
    print(f"Processing {len(old_data)} players...")
    
    # Get all unique UUIDs
    unique_uuids = list(set(old_data.values()))
    print(f"Found {len(unique_uuids)} unique UUIDs")
    
    # Fetch current usernames
    uuid_to_name = {}
    
    def fetch_with_delay(uuid):
        time.sleep(delay)
        name = get_username_from_uuid(uuid)
        return uuid, name
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_uuid = {executor.submit(fetch_with_delay, uuid): uuid for uuid in unique_uuids}
        
        for i, future in enumerate(as_completed(future_to_uuid)):
            uuid, current_name = future.result()
            if current_name:
                uuid_to_name[uuid] = current_name
                print(f"Progress: {i+1}/{len(unique_uuids)} - {current_name}")
            else:
                print(f"Progress: {i+1}/{len(unique_uuids)} - Failed to get name for {uuid}")
    
    # Create updated data structure
    updated_data = {}
    name_changes = []
    
    for old_name, uuid in old_data.items():
        current_name = uuid_to_name.get(uuid)
        if current_name:
            updated_data[current_name] = uuid
            if old_name != current_name:
                name_changes.append((old_name, current_name, uuid))
        else:
            # Keep old entry if we couldn't fetch current name
            updated_data[old_name] = uuid
            print(f"Warning: Keeping old name '{old_name}' - couldn't fetch current name")
    
    # Save updated data
    with open(output_file, 'w') as f:
        json.dump(updated_data, f, indent=2)
    
    # Print summary
    print(f"\nUpdate complete!")
    print(f"Total players: {len(updated_data)}")
    print(f"Name changes detected: {len(name_changes)}")
    print(f"Updated data saved to: {output_file}")
    
    if name_changes:
        print("\nName changes:")
        for old_name, new_name, uuid in name_changes:
            print(f"  {old_name} -> {new_name} ({uuid})")
    
    return updated_data, name_changes

def create_name_list(data, output_file):
    """Create a simple list of current player names"""
    names = sorted(data.keys())
    
    with open(output_file, 'w') as f:
        for name in names:
            f.write(f"{name}\n")
    
    print(f"Player name list saved to: {output_file}")
    return names

if __name__ == "__main__":
    # Update the player data
    updated_data, changes = update_player_names(
        'player_uuids.json', 
        'player_uuids.json'
    )
    
    # Create a simple name list
    names = create_name_list(updated_data, 'player_names.txt')
    
    print(f"\nCurrent player names ({len(names)} total):")
    for i, name in enumerate(names[:10]):  # Show first 10
        print(f"  {name}")
    if len(names) > 10:
        print(f"  ... and {len(names) - 10} more")

Processing 241 players...
Found 241 unique UUIDs
Progress: 1/241 - chrisihalt
Progress: 2/241 - Dubbly
Progress: 3/241 - Harold_Sensemann
Progress: 4/241 - Laaaachs
Progress: 5/241 - Oronor
Progress: 6/241 - HerrDante
Progress: 7/241 - Teesily
Progress: 8/241 - Freeeedom
Progress: 9/241 - juvona
Progress: 10/241 - Gummibearchen
Progress: 11/241 - Bikop
Progress: 12/241 - Carl1_1
Progress: 13/241 - Feinberg
Progress: 14/241 - Kanickelul
Progress: 15/241 - Juti0n
Progress: 16/241 - 2wc
Progress: 17/241 - lvlaurin
Progress: 18/241 - Gestimus
Progress: 19/241 - Highsay
Progress: 20/241 - Falke23_5
Progress: 21/241 - Martomias
Progress: 22/241 - lolisamasan
Progress: 23/241 - ScreamingBlatz
Progress: 24/241 - Falke_01
Progress: 25/241 - Kopfradio
Progress: 26/241 - cheesecake2901
Progress: 27/241 - Steavn
Progress: 28/241 - GingerTeddy
Progress: 29/241 - LeWi_100
Progress: 30/241 - Anni808
Progress: 31/241 - JOW24
Progress: 32/241 - Allooy
Progress: 33/241 - Joel_the_king
Progress: 34/241 -

### old approach to generating playerList (depreciated)

import pyautogui
import os
from tqdm import tqdm, trange

pyautogui.FAILSAFE = True

# check OS 
userOS = os.name

if "nt" in userOS: 
    directory = os.getcwd() + "\\in-new"
elif "posix" in userOS:
    directory = os.getcwd() + "/in-new"
else: 
    directory = None

file_names = []

for file in os.listdir(directory):
    if file.endswith(".txt") and "log" not in file:
        file_names.append(file[:-4])

playerList = sorted(file_names)
renamed = ["BacardiSprite", "jimmi1410timeter", "DerBaumIstDerWeg", "CML_Justin", "DaHinige_", "ryzonabi", "badinfluence666", "officialrzyon", "187leonbande", "Bannbaar", "BearchenGott", "H1ghsay", "Kosmaxnaut", "Livgo", "Lubotter2009", 'JustK1lling', 'schmaarek', "ymq_", "zTotito", "Radiokopf", "toxicplace", "KMaxN", "Liichtenstein"]
for i in renamed:   
    try: playerList.remove(i)
    except: ValueError(i + " not in list.")

newPlayers = ["RasenLP", "FluffyDragon2007", "Tim_Voltia", "BunnyKiko", "BauHD", "Joe3346", "rotmann2", "TAMISH009", "Muniix", "redbrll"]

playerList.extend(newPlayers)

playerLower = [i.lower() for i in playerList]

playerList = sorted(playerList)

playerList = sorted(list(dict.fromkeys(playerList)))

# Check for .txt files in the desktop path and add them to a list, then remove .txt from each string and remove all matching entries from playerList
desktop_path = os.path.expanduser("~/Desktop")
desktop_files = os.listdir(desktop_path)


for file in desktop_files:
    if file.endswith(".txt"):
        file = file[:-4]
    
    try: playerList.remove(file)
    except: pass

# Remove duplicates from playerList
playerList = list(dict.fromkeys(playerList))

# remove "log" and "ItemTooltipExport" from playerList
try: playerList.remove("log")
except: pass
try: playerList.remove("itemTooltipExport")
except: pass


# check for all filenames on the Desktop and remove from playerList (non case sensitive)
for file in desktop_files:
    if file.endswith(".txt"):
        file = file[:-4]
    try: playerList.remove(file)
    except: pass

print(playerList)

### new (just using api-generated name list from UUIDs in player_uuids.json)

In [46]:
playerList = names

CONSIDER_PROGRESS = True

if CONSIDER_PROGRESS:
    # Check for .txt files in the desktop path and add them to a list, then remove .txt from each string and remove all matching entries from playerList
    desktop_path = os.path.expanduser("~/Desktop")
    desktop_files = os.listdir(desktop_path)


    for file in desktop_files:
        if file.endswith(".txt"):
            file = file[:-4]
        
        try: playerList.remove(file)
        except: pass

    # Remove duplicates from playerList
    playerList = list(dict.fromkeys(playerList))

    # remove "log" and "ItemTooltipExport" from playerList
    try: playerList.remove("log")
    except: pass
    try: playerList.remove("itemTooltipExport")
    except: pass


    # check for all filenames on the Desktop and remove from playerList (non case sensitive)
    for file in desktop_files:
        if file.endswith(".txt"):
            file = file[:-4]
        try: playerList.remove(file)
        except: pass

print("playerList contains " + str(len((playerList)))+ " unique names")

playerList contains 241 unique names


In [47]:
"Allooy" in playerList

True

### Scraping function

In [48]:
import keyboard
import pyautogui
import keyboard
from tqdm import trange


def scrape(resolution = "2k", fast=False, extended=False, dur=None):

    # Scrape Records for Each Player in playerList
    for i, player in enumerate(playerList):

        if sorted(playerList) == sorted(desktop_files): 
            print("All records have been scraped.")
            break

        with trange(len(playerList), desc="Processing players", position=0) as t:
            t.update(i)

            # Open Chat
            pyautogui.press("t")
            pyautogui.sleep(0.2)

            # Assemble Command String
            command = "/rekorde" + " " + player

            # Execute Command
            #pyautogui.typewrite(command, 
            #                    #interval=0.01
            #                    )
            pyautogui.sleep(0.2)
            keyboard.write(command)
            pyautogui.sleep(0.2)

            pyautogui.press("enter")

            pyautogui.sleep(0.15)

            # mooth smoothly through all positions x, y, taking x and y from the respective list

            pyautogui.moveTo(x = 500, y = 500, duration=0.002)

            if resolution == "4k": coordTpls = _4krecordIconPositions
            elif resolution == "2k": coordTpls = recordIconPositions
            elif resolution == "1080p": coordTpls = recordIconPositions1080p
            if extended: coordTpls = recordIconPositionsExtended
            else: coordTpls = recordIconPositions

            if fast or True: 
                coordTpls = fastIconPositions

            duration = 0.7 if not fast else 0.05

            if dur is not None: duration = float(dur)

            if fast or dur is not None: sleep = duration

            for i in range(0, (len(coordTpls))):
                pyautogui.moveTo(x = coordTpls[i], duration=duration)
                # pyautogui.click()
                pyautogui.sleep(sleep)

            pyautogui.press("e")

            # Wait 200 ms until next player] 
            pyautogui.sleep(0.2)

            # Rename file on desktop
            new_filename = player + ".txt"
            desktop_path = os.path.expanduser("~/Desktop")
            old_filepath = os.path.join(desktop_path, "ItemTooltipExport.txt")
            new_filepath = os.path.join(desktop_path, new_filename)
            try: os.rename(old_filepath, new_filepath)
            except: print(f"Error renaming file for {player}.")

            # if log: print(f"Record for {player} <{i+1}> scraped successfully.\n")

def verifyDesktop():
    # verify the linecount for each file, if any file has more or less than 21 lines, print the filename, else give an OK statement, if there is no file for a player, also point that out. List all players in each "category" at once, not as individual line
    for player in playerList:
        try:
            with open(os.path.join(desktop_path, player + ".txt"), "r") as file:
                try: 
                    lines = file.readlines()
                    if len(lines) != 27:
                        print(player + " has " + str(len(lines)) + " lines.")
                except: pass
        except FileNotFoundError:
            print(player + " has no file.")
    print("All files checked.")

def clear():
    try: os.remove(os.path.join(desktop_path, "ItemTooltipExport.txt"))
    except: pass
    try: os.remove(os.path.join(desktop_path, "log.txt"))
    except: pass

In [49]:
# clear()

pyautogui.sleep(5)

scrape(fast=False, dur=0.002, resolution="2k")

pyautogui.sleep(2)

clear()

verifyDesktop()

Processing players:  80%|████████  | 194/241 [00:04<00:01, 44.56it/s]


Error renaming file for cheesecake2901.


Processing players:  81%|████████  | 195/241 [00:04<00:01, 44.66it/s]


Error renaming file for chrisihalt.


Processing players: 100%|█████████▉| 240/241 [00:04<00:00, 54.92it/s]


20LeRe10 has 24 lines.
2B9 has 24 lines.
2wc has 24 lines.
Acidey has 24 lines.
Allooy has 24 lines.
Anni808 has 24 lines.
AnzeigeGehtRaus has 24 lines.
ArthurAlchemist has 24 lines.
Axollotel has 24 lines.
Bartschii has 24 lines.
BastiGHG has 24 lines.
BauHD has 24 lines.
Bikoop has 24 lines.
Bikop has 24 lines.
Bildungsarten has 24 lines.
BunnyKiko has 24 lines.
BusinessBent has 24 lines.
ByNetherdude has 24 lines.
CML_Justin has 24 lines.
Carl1_1 has 24 lines.
Chander24 has 24 lines.
CladyNoClip_ has 24 lines.
Cytoox has 24 lines.
D4rkCookie has 24 lines.
DaVinci_Son has 24 lines.
DarkCobweb has 24 lines.
Dat_Klan has 24 lines.
DerAutist has 24 lines.
DieserBear has 24 lines.
Dokkkkko has 24 lines.
DrSpeed06 has 24 lines.
Dubbly has 24 lines.
EinfachEazy has 24 lines.
Ex4cted has 24 lines.
Falke23_5 has 24 lines.
Falke_01 has 24 lines.
Feinberg has 24 lines.
Fflopse has 24 lines.
Flitzi_Dino has 24 lines.
FluffyDragon2007 has 24 lines.
FlyingAutismus has 24 lines.
FlyingKyubi has 24